In [7]:
import torch
import ase
from ase.io import read, write
import numpy as np
import scipy
from ase.build import molecule
from ase.neighborlist import neighbor_list


In [2]:
from torch_nl.ase_impl import ase_neighbor_list

# test n^2 impl

In [14]:
frame = molecule('OCHCHO')
rcut = 3

pos = torch.from_numpy(frame.get_positions())
cell = torch.from_numpy(frame.get_positions())
pbc = torch.from_numpy(frame.get_pbc())

In [15]:
idx_i, idx_j, idx_S, dist = neighbor_list(
        "ijSd", frame, cutoff=rcut, self_interaction=False
    )
np.sort(dist)

array([1.10563938, 1.10563938, 1.10563938, 1.10563938, 1.22295086,
       1.22295086, 1.22295086, 1.22295086, 1.51286   , 1.51286   ,
       2.05241169, 2.05241169, 2.05241169, 2.05241169, 2.22229623,
       2.22229623, 2.22229623, 2.22229623, 2.38769877, 2.38769877,
       2.38769877, 2.38769877, 2.61851372, 2.61851372, 2.61851372,
       2.61851372])

In [ ]:
def fully_connected2torch(i_ids, j_ids) -> torch.Tensor:
    mapping = torch.cartesian_prod(i_ids, j_ids).t()
    mapping = mapping[:, mapping[0] != mapping[1]]
    return mapping